# Resolução da lista 2 de NLP
## Alunos:
    - Eduardo Brasil Araujo
    - Gideão Pinheiro

In [128]:
import pandas as pd
import numpy as np

# Questão 1

# Questão 2

In [129]:
data = pd.read_csv('../datasets/website_classification.csv')

In [367]:
X_train = data.cleaned_website_text

In [368]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1,1),
                                   stop_words='english',
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_features=1000)

np.random.seed(123)

np.random.shuffle(np.array(X_train))

X = count_vectorizer.fit_transform(X_train)

In [273]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

search_params = {
  'n_components': list(range(1, 11)),
}

lda = LatentDirichletAllocation()

grid_search = GridSearchCV(lda,
                           param_grid=search_params,
                           n_jobs=4,
                           verbose=1,
                           cv=4)
grid_search.fit(X)

print(f'Best model\'s Params: {grid_search.best_params_}')
print(f'Best log likelihood score: {grid_search.best_score_}')

Fitting 4 folds for each of 10 candidates, totalling 40 fits
Best model's Params: {'n_components': 5}
Best log likelihood score: -889269.6344358076


### Respondendo item (a) da 2ª questão:

- Referência: https://investigate.ai/text-analysis/choosing-the-right-number-of-topics-for-a-scikit-learn-topic-model/

É notório que o modelo LDA é melhor para realizar a avaliação de quantos
tópicos utilizar, pois ele proporciona uma métrica de avaliação bem definida,
diferentemente de outros modelos como NMF (pelo menos no scikit-learn).

Assim, foi utilizado este modelo para avaliar a quantidade de tópicos, e
então, foi feito uma busca em grade pela combinação de parâmetros mais bem
adaptados ao problema; com o GridSearchCV. 

O melhor resultado foi com o número de componentes (número de tópicos)
igual a 5. Porém, como pode ser observado no item b desta mesma questão,
na tabela mostrada abaixo dos tópicos foram observadas alguma anomalias.
A tabela em questão é uma distribuição dos documentos em relação aos
tópicos gerados, ou seja, o número representa a incidência dos tópicos em
relação à base. A primeira anomalia observada é em SVD, com os valores
sendo negativos. A segunda anomalia observada é o NMF com uma coluna com
todos os valores zerados. As anomalias observadas são facilmente
resolvidas diminuindo a quantidade de tópicos para 4, que, logo, será a
quantidade final de tópicos escolhidos.

In [262]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

In [369]:
# Value
number_of_components = 4

# LDA
lda = LatentDirichletAllocation(n_components=number_of_components)
lda_topics = lda.fit_transform(X)

# SVD
svd = TruncatedSVD(n_components=number_of_components)
svd_topics = svd.fit_transform(X)

# NMF
nmf = NMF(n_components=number_of_components)
nmf_topics = nmf.fit_transform(X)

In [364]:
def get_topic_list(model, feature_names, n_words):
    topic_list = []
    for topic_idx, topic in enumerate(model.components_):
        top_n = ['_'.join(feature_names[i].split())
                for i in topic.argsort()
                [-n_words:]][::-1]
        top_features = ' '.join(top_n)
        topic_list.append(f"topic_{'_'.join(top_n[:3])}") 

        print(f"Topic {topic_idx}: {top_features}")
    return topic_list

### Respostas ao item (b) da 2ª questão:

- Referência: https://www.freecodecamp.org/news/advanced-topic-modeling-how-to-use-svd-nmf-in-python/

In [370]:
# LDA
top_feature_names = count_vectorizer.get_feature_names_out()

topic_list = get_topic_list(lda, top_feature_names, 5)

amounts = lda.transform(X) * 100
topics = pd.DataFrame(amounts, columns=topic_list)
topics.head()

Topic 0: news december world new league
Topic 1: information use new service work
Topic 2: chat tv free sex movie
Topic 3: recipe new add gift view


,topic_news_december_world,topic_information_use_new,topic_chat_tv_free,topic_recipe_new_add
0,0.066607,18.213757,0.065194,81.654443
1,0.061580,17.872623,28.812409,53.253388
2,0.138746,35.743256,0.143735,63.974262
3,0.535152,0.032640,0.031903,99.400305
4,0.068734,33.046922,26.194239,40.690105


In [371]:
# SVD
topic_list = get_topic_list(svd, top_feature_names, 5)

amounts = svd.transform(X) * 100
topics = pd.DataFrame(amounts, columns=topic_list)
topics.head()

Topic 0: news late trend new world
Topic 1: sex phone add cart free
Topic 2: add cart quality good fresh
Topic 3: tv movie series comedy tvma


,topic_news_late_trend,topic_sex_phone_add,topic_add_cart_quality,topic_tv_movie_series
0,421.204588,470.426578,321.653989,732.434358
1,565.180837,716.901669,293.254237,980.098028
2,253.582559,315.719082,222.888538,424.663615
3,1005.060191,1106.810230,732.593426,1520.173618
4,469.457457,554.920273,375.323692,809.873944


In [372]:
# NMF
topic_list = get_topic_list(nmf, top_feature_names, 5)

amounts = nmf.transform(X) * 100
topics = pd.DataFrame(amounts, columns=topic_list)
topics.head()

Topic 0: news late trend india new
Topic 1: sex phone free hot gay
Topic 2: add cart quality fresh good
Topic 3: tv movie series comedy tvma


,topic_news_late_trend,topic_sex_phone_free,topic_add_cart_quality,topic_tv_movie_series
0,2.866815,1.825468,6.045565,33.891438
1,3.838170,6.576929,5.246591,45.258195
2,1.626564,1.636281,5.058541,20.275164
3,8.544271,6.026568,15.552746,72.585468
4,3.004599,2.505878,7.496795,38.040051


### Resposta do item (c) da 2ª questão:

In [373]:
# Choosing documents
indexes = np.random.choice(range(len(data)), 5)
choosen_documents = []
choosen_documents_categories = []
for index in indexes:
    choosen_documents_categories.append(data.Category[index])
    choosen_documents.append(data.cleaned_website_text[index])
choosen_documents = np.array(choosen_documents)
choosen_documents_categories = np.array(choosen_documents_categories)

In [376]:
num_topics = 3

count_vectorizer_3 = CountVectorizer(ngram_range=(1,1),
                                   stop_words='english',
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_features=1000)

choosen_documents_converted = count_vectorizer.fit_transform(choosen_documents)

# LDA
lda_3 = LatentDirichletAllocation(n_components=num_topics)
lda_3.fit(choosen_documents_converted)

# SVD
svd_3 = TruncatedSVD(n_components=num_topics)
svd_3.fit(choosen_documents_converted)

# NMF
nmf_3 = NMF(n_components=num_topics)
nmf_3.fit(choosen_documents_converted)

NMF(n_components=3)

In [377]:
feature_names = count_vectorizer.get_feature_names_out()

print('LDA Topics')
lda_topics_str = get_topic_list(lda_3, feature_names, 5)
print('SVD Topics')
svd_topics_str = get_topic_list(svd_3, feature_names, 5)
print('NMF Topics')
nmf_topics_str = get_topic_list(nmf_3, feature_names, 5)

LDA Topics
Topic 0: mayo clinic condition botany letter
Topic 1: base visit videos overview blog
Topic 2: date somaiya single site dating
SVD Topics
Topic 0: mayo clinic condition letter begin
Topic 1: botany plant science somaiya life
Topic 2: somaiya programme place student life
NMF Topics
Topic 0: mayo clinic condition letter begin
Topic 1: botany plant science math tech
Topic 2: somaiya place programme life student


In [378]:
# Escrevendo documento para ler os dados
with open('output_file.txt', 'wb') as fp:
    for outer_index, line in enumerate(choosen_documents.tolist()):
        fp.write(f'\n{"#" * 30}\n'.encode('utf-8'))
        fp.write(f'\n{choosen_documents_categories[outer_index]}\n\n'.encode('utf-8'))
        for index, word in enumerate(line.split()):
            fp.write(f'{word} '.encode('utf-8'))
            if (index + 1) % 12 == 0:
                fp.write('\n'.encode('utf-8'))

### Resposta do item (d) da 2ª questão:

- Referência: https://stackoverflow.com/questions/60613532/how-do-i-calculate-the-coherence-score-of-an-sklearn-lda-model#62151839

In [216]:
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora

In [222]:
def get_Cv(model, df_columnm):
  topics = model.components_

  n_top_words = 20
  texts = [[word for word in doc.split()] for doc in df_columnm]

  # create the dictionary
  dictionary = corpora.Dictionary(texts)
  # Create a gensim dictionary from the word count matrix

  # Create a gensim corpus from the word count matrix
  corpus = [dictionary.doc2bow(text) for text in texts]

  feature_names = [dictionary[i] for i in range(len(dictionary))]

  # Get the top words for each topic from the components_ attribute
  top_words = []
  for topic in topics:
      top_words.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])

  coherence_model = CoherenceModel(topics=top_words, texts=texts, dictionary=dictionary, coherence='c_v')
  coherence = coherence_model.get_coherence()
  return coherence

In [379]:
lda_coherence = get_Cv(lda, X_train)
svd_coherence = get_Cv(svd, X_train)
nmf_coherence = get_Cv(nmf, X_train)

In [380]:
print(lda_coherence)
print(svd_coherence)
print(nmf_coherence)

0.4469203505209559
0.4439693854804786
0.42701229768501564


In [381]:
lda_3_coherence = get_Cv(lda_3, choosen_documents)
svd_3_coherence = get_Cv(svd_3, choosen_documents)
nmf_3_coherence = get_Cv(nmf_3, choosen_documents)

In [382]:
print(lda_3_coherence)
print(svd_3_coherence)
print(nmf_3_coherence)

0.6971816854566728
0.7134016171306804
0.7126874769842947


### Justificativa

É notório que os resultados dos três modelos foram similares, tanto numa análise
manual, com a leitura de cada um deles. Como também com a métrica de coerência
(métrica que avalia a interpretabilidade dos tópicos) apresenta-se altamente
similar, mas que, ao mesmo tempo, percebe-se que o modelo SVD se saiu bem em
ambos os testes (o com a base toda e outro com somente 5 documentos). Portanto,
o SVD é escolhido como o que melhor se desempenhou, contudo os resultados são
praticamente os mesmos.

# Questão 3

# Questão 4